In [1]:
import openai

In [2]:
import minsearch

In [3]:
import json

In [4]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
# keyword_fields are used for filtering
# perform search on question text and section
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [8]:
from openai import OpenAI
client = OpenAI()

In [9]:
def search(query):
    boost = {'question' : 3.0, 'section' : 0.5}
    results = index.search(
        query = query,
        filter_dict = {'course' : 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )
    return results

In [10]:
def build_prompt(query, search_results):
    prompt_template = """ 
You are a course teaching assistant. Answer the QUESTION based on CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesnt contain the answer, output None.

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']} \nquestion: {doc['question']} \ntext: {doc['text']}"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [11]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', "content": prompt}]
    )

    return response.choices[0].message.content

In [12]:
query = "the course has already started, can I still enroll?"
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [13]:
rag(query)

'Yes, even if the course has already started, you can still enroll and submit the homeworks. However, be aware of the deadlines for turning in the final projects.'

In [17]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")

In [32]:
es_client.info()

ObjectApiResponse({'name': '406059c132fb', 'cluster_name': 'docker-cluster', 'cluster_uuid': '79VBwse_QLyMHw8gIZDDRw', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [33]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [34]:
# Iterate over documents [] and insert into ElasticSearch index we just created
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [22]:
query

'the course has already started, can I still enroll?'

In [42]:
# As you see, question is given 3 times importance here compared to text and section. 
# We filter and limit to data-engineering-zoomcamp
def search_elastic(query, filter_course="data-engineering-zoomcamp"):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": filter_course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    
    for hit in response["hits"]["hits"]:
        result_docs.append(hit['_source'])
    
    return result_docs

In [43]:
result_docs = search_elastic(query)

In [44]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [45]:
query

'the course has already started, can I still enroll?'

In [48]:
def rag_with_elastic(query):
    search_results = search_elastic(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [49]:
rag_with_elastic(query)

'Yes, you can still enroll and submit homework even if the course has already started.'